<a href="https://colab.research.google.com/github/Yoshiki0418/Deep_Learning/blob/main/DL_Lecture2/MACs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MACs

In [1]:
import torch
from torch import nn

### MACsをスクラッチで計算する

In [2]:
def calc_output_shape(layer, input_shape):
    input = torch.randn(input_shape)
    output = layer(input)
    return tuple(output.shape)

In [7]:
def calc_macs_conv2d(layer, input_shape):
    b, in_ch, in_h, in_w = input_shape
    out_ch, _, f_h, f_w = layer.weight.shape

    _, _, out_h, out_w = calc_output_shape(layer, input_shape)

    macs = b * in_ch * out_ch * f_h * f_w * out_h * out_w
    return macs

def calc_macs_linear(layer, input_shape):
    b, in_features = input_shape
    out_features = layer.weight.shape[0]

    macs = b * in_features * out_features
    return macs

In [4]:
input_shape = (1, 1, 128, 128)
X = torch.randn(input_shape)
# 畳み込み層
conv_layer = nn.Conv2d(1, 8, kernel_size=3)
calc_macs_conv2d(conv_layer, input_shape)

1143072

In [8]:
# 全結合層
linear_layer = nn.Linear(64, 10)
input_shape = (1, 64)
calc_macs_linear(linear_layer, input_shape)

640

### thopでMACsを求める

In [11]:
!pip install thop

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [12]:
import thop

# conv2d
macs, params = thop.profile(conv_layer, (X,))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.


In [13]:
# スクラッチの関数で求めたMACsと等しいことを確認
macs

1143072.0

In [14]:
# linear
X = torch.randn(input_shape)
macs, params = thop.profile(linear_layer, (X,))
# スクラッチの関数で求めたMACsと等しいことを確認
macs

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


640.0